In [2]:
import os
 

import numpy as np
import pandas as pd
from flask import Flask, request, jsonify, render_template
import pickle

app = Flask(__name__)
model = pickle.load(open('model.pkl', 'rb'))

In [3]:
dataset = pd.read_csv('adult.csv')

In [4]:
dataset

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
48838,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
48839,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
48840,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [7]:
dataset['income'] = le.fit_transform(dataset['income'])

In [8]:
dataset = dataset.replace('?', np.nan)

In [17]:
columns_with_nan = ['workclass','occupation', 'native-country']

In [18]:
for col in columns_with_nan:
    dataset[col].fillna(dataset[col].mode()[0], inplace=True)

C:\Users\rakes\AppData\Local\Temp\ipykernel_14876\3965776787.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset[col].fillna(dataset[col].mode()[0], inplace=True)


In [19]:
print(dataset.columns)

Index(['age', 'workclass', 'fnlwgt', 'education', 'educational-num',
       'marital-status', 'occupation', 'relationship', 'race', 'gender',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'income'],
      dtype='object')


In [20]:
for col in dataset.columns:
    if dataset[col].dtypes == 'object':
        encoder = LabelEncoder()
        dataset[col] = encoder.fit_transform(dataset[col])

In [21]:
X = dataset.drop('income', axis=1)
Y = dataset['income']

In [22]:
X = X.drop(['workclass','education','race','gender','capital-loss','native-country','fnlwgt','relationship','capital-gain'], axis=1) 

In [24]:
from sklearn.preprocessing import StandardScaler

In [35]:
X = scaler.fit_transform(X)

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict',methods=['POST'])
def predict():

     if request.method == 'POST':
         marital_name = request.form['marital-status']

         marital_value = 0

         if marital_name == 'Married-civ-spouse':
             marital_value = 1
         elif marital_name == 'Never-married':
             marital_value = 2
         elif marital_name == 'Divorced':
             marital_value = 3 
         elif marital_name == 'Separated':
             marital_value = 4
         elif marital_name == 'Widowed':
             marital_value = 5
         elif marital_name == 'Married-Spouse-absent':
             marital_value = 6
         elif marital_name == 'Married-AF-spouse':
             marital_value = 7

        age_value = request.form['age']
        edu_num_value = request.form['Year Of Education']
        occupation_value = request.form['Occupation Code']
        hours_value = request.form['Hours Of Work Per Week']

    features = [age_value, edu_num_value, marital_value,
                occupation_value, hours_value]

    int_features = [int(x) for x in features]
    final_features = [np.array(int_features)]
    prediction = model.predict(scaler.transform(final_features))
    
    if rediction == 1:
        output = "Income Is More Than 50k"
    elif prediction == 0:
        output = "Income Is Less Than 50K"

    return render_template('index.html',prediction_text='{}'.format(output))
        
if __name__ == "__main__":
    app.run(debug=True)